In [13]:
# importing libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import openpyxl as op

In [14]:
# scrape data

def findByInning(keyword, group):
    url = 'http://www.cricmetric.com/playerstats.py?player='+keyword+'&role=batsman&format=Test&groupby='+group
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, 'lxml')
    table = soup.find('div', class_='tab-content')
    rows = table.find_all('tr')
    return rows

In [15]:
# get headers

def findHeaders(rows):
    headers=[]
    vals = rows[0].find_all('th')
    for i in range(0,len(vals)):
        val = vals[i].get_text()
        headers.append(val)
    return headers

In [16]:
# get table data

def findData(rows):
    data = []
    for i in range(1,len(rows)):
        subData = []
        vals = rows[i].find_all('td')
        for i in range(0,len(vals)):
            val = vals[i].get_text()
            subData.append(val)
        data.append(subData)
    return data

In [17]:
# create dataframe

def createTable(keyword, group):
    rows = findByInning(keyword, group)
    headers = findHeaders(rows)
    data = findData(rows)
    df = pd.DataFrame(data=data, columns=headers)
    df['Key'] = keyword
    return df

In [18]:
#merge dataframes for all players

players = ['SPD+Smith', 'V+Kohli', 'KS+Williamson', 'JE+Root']
groups = ['year', 'inning', 'batpos', 'opp_player', 'opp_player_type', 'opp_team', 'match_result', 'vcountry']

path = 'E:/Datasets/ScrapedDatasets/FabFour.xlsx'
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

for group in groups:
    dfSmith = createTable(players[0], group)
    dfKohli = createTable(players[1], group)
    dfKane = createTable(players[2], group)
    dfRoot = createTable(players[3], group)
    final_df = pd.concat([dfSmith, dfKohli, dfKane, dfRoot], ignore_index=True)   
    final_df.to_excel(writer, index=False, sheet_name='by-'+group)

writer.close()
print('done')

done
